"Welcome to this PySpark data cleansing tutorial! In this video, we’ll take a messy, real-world dataset and transform it into clean, ready-to-use data using just a few powerful PySpark techniques."

First, we import SparkSession—this starts our Spark engine and manages our data.
Next, we bring in functions as F, a toolbox full of built-in functions for cleaning and transforming columns.
And finally, we import Window as W, which lets us create advanced analytics like ranking, deduplication, and time-based filtering.

In [0]:

# Import SparkSession, SQL functions, and Window specification from PySpark

from pyspark.sql import SparkSession, functions as F
from pyspark.sql import Window as W

We start by creating a Spark session. This is like launching our data workshop—it controls the whole PySpark process.

In [0]:
#1)Initialize Spark Session

spark = (SparkSession.builder              # Initialize a SparkSession builder
.appName("DataCleaningDemo")               # Set the Spark application name
.config("spark.sql.session.timeZone", "UTC")  # Configure the Spark SQL session timezone to UTC
.getOrCreate())                            # Create a new SparkSession or get the existing one

Next, we load our messy CSV dataset, which contains duplicates, missing values, inconsistent formats, and even some extreme outliers.

In [0]:
# 2) Load Messy Dataset with Anomalies
df = (spark.read                           # Start reading data using Spark DataFrameReader
.option("header", True)                    # Treat the first row as header
.option("multiLine", True)                 # Allow multiline fields in the CSV
.option("escape", '"')                     # Use double quotes to escape special characters
.csv("/Volumes/workspace/csv_files/csv_files/dataset.csv"))  # Load CSV file from specified path

We always start by inspecting the raw data with .show() and .printSchema(). This helps us see what we’re dealing with.

In [0]:
# 3) Inspect Data
df.show(10, truncate=False)  # Display first 10 rows without truncating column values
df.printSchema()             # Print the schema of the DataFrame with column types

+---+------+---------+---------+---------------+------+---------------+
|ID | Name |Amount($)|is Active|Order Date     |Sex   |Updated At     |
+---+------+---------+---------+---------------+------+---------------+
|152|Ånna  | 1 200   |Inactive |2/26/2023 22:12|FEMALE|6/1/2023 16:02 |
|415|Renée |10.000,00|1        |1/15/2023 10:57|f     |2/13/2023 19:50|
|119|Judy  |1,000.50 |Active   |6/2/2023 17:14 |M     |4/6/2023 15:32 |
|365|Ivan  |-5000    |N        |2/2/2023 21:16 |f     |2/15/2023 21:07|
|30 |Bob   |NA       |TRUE     |NULL           |Other |1/13/2023 1:20 |
|403|Frank |500      |Active   |3/31/2023 0:58 |f     |4/6/2023 5:51  |
|64 |David |2000     |NO       |1/16/2023 21:43|f     |6/4/2023 1:03  |
|482|Alice |2.000,50 |Y        |6/10/2023 16:42|FEMALE|6/10/2023 20:17|
|442|Hannah|10.000,00|Inactive |2/8/2023 14:43 |F     |2/4/2023 7:50  |
|448|Renée |2.000,50 |Active   |5/1/2023 17:50 |f     |1/22/2023 2:21 |
+---+------+---------+---------+---------------+------+---------

Then, we standardize column names—removing spaces, special characters, and making them lowercase. Clean columns make everything easier downstream.

In [0]:
# 4) Standardize Column Names

import re                                             # Import regular expressions module
new_cols = []                                         # Initialize list to store cleaned column names
for c in df.columns:                                  # Iterate over each column name in the DataFrame
    nc = re.sub(r"[\s\-]+", "_", c.strip().lower())   # Replace spaces/hyphens with underscores and convert to lowercase
    nc = re.sub(r"[^0-9a-z_]+", "", nc)               # Remove any characters that are not alphanumeric or underscores
    nc = re.sub(r"_+", "_", nc).strip("_")            # Replace multiple underscores with single and strip leading/trailing underscores
    new_cols.append(nc)                               # Append cleaned column name to the list
for old, new in zip(df.columns, new_cols):            # Pair old and new column names for renaming
    if old != new:                                    # Check if the column name has changed
        df = df.withColumnRenamed(old, new)           # Rename the column in the DataFrame

df.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- amount: string (nullable = true)
 |-- is_active: string (nullable = true)
 |-- order_date: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- updated_at: string (nullable = true)



Here we trim spaces, remove non-breaking spaces, and replace placeholder values like ‘NA’ or dashes with proper nulls.

In [0]:
# 5) Clean Missing Values & Strings

na_tokens = ["", "na", "n/a", "none", "null", "-", "--", "unknown"]  # Define tokens representing missing values
for c, t in df.dtypes:                                               # Iterate through each column and its data type
    if t == 'string':                                                # Process only string columns
        df = df.withColumn(c, F.regexp_replace(F.col(c), "\xa0", " "))  # Replace non-breaking spaces with regular spaces
        df = df.withColumn(c, F.trim(F.col(c)))                         # Trim leading and trailing spaces
        df = df.withColumn(c, F.regexp_replace(F.col(c), r"\s+", " "))  # Replace multiple spaces with a single space
        df = df.withColumn(                                           
            c,                                                        # For the current column
            F.when(F.lower(F.col(c)).isin(na_tokens), None)           # Replace NA tokens with null
             .otherwise(F.col(c))                                     # Keep the original value otherwise
        )

df.show(10, truncate=False)

+---+------+---------+---------+---------------+------+---------------+
|id |name  |amount   |is_active|order_date     |sex   |updated_at     |
+---+------+---------+---------+---------------+------+---------------+
|152|Ånna  |1 200    |Inactive |2/26/2023 22:12|FEMALE|6/1/2023 16:02 |
|415|Renée |10.000,00|1        |1/15/2023 10:57|f     |2/13/2023 19:50|
|119|Judy  |1,000.50 |Active   |6/2/2023 17:14 |M     |4/6/2023 15:32 |
|365|Ivan  |-5000    |N        |2/2/2023 21:16 |f     |2/15/2023 21:07|
|30 |Bob   |NULL     |TRUE     |NULL           |Other |1/13/2023 1:20 |
|403|Frank |500      |Active   |3/31/2023 0:58 |f     |4/6/2023 5:51  |
|64 |David |2000     |NO       |1/16/2023 21:43|f     |6/4/2023 1:03  |
|482|Alice |2.000,50 |Y        |6/10/2023 16:42|FEMALE|6/10/2023 20:17|
|442|Hannah|10.000,00|Inactive |2/8/2023 14:43 |F     |2/4/2023 7:50  |
|448|Renée |2.000,50 |Active   |5/1/2023 17:50 |f     |1/22/2023 2:21 |
+---+------+---------+---------+---------------+------+---------

Amounts become numbers, booleans become true or false, and dates are parsed into proper timestamps—even with mixed formats.

In [0]:
# 6) Convert Data Types

# Amount to Double (handle full-width digits)

df = df.withColumn("amount", F.regexp_replace("amount", "０", "0"))             # Replace full-width zero with normal zero
df = df.withColumn("amount", F.regexp_replace("amount", r"[^0-9,\.-]", ""))     # Remove all non-numeric, non-comma, non-dot, non-dash characters
df = df.withColumn("amount", F.regexp_replace("amount", r"\.", ""))             # Remove periods used as thousands separators
df = df.withColumn("amount", F.regexp_replace("amount", r",", "."))             # Replace comma with dot for decimal conversion
df = df.withColumn("amount", F.col("amount").cast("double"))                    # Cast cleaned amount column to double type

# Boolean

df = df.withColumn(
    "is_active",
    F.when(F.lower("is_active").isin(["true","t","yes","y","1","active"]), F.lit(True))    # Map common truthy values to True
     .when(F.lower("is_active").isin(["false","f","no","n","0","inactive"]), F.lit(False)) # Map common falsy values to False
     .otherwise(F.lit(None).cast("boolean"))                                               # Set all other/unrecognized values to NULL
)

# Dates

# order_date

from pyspark.sql.functions import col, trim, regexp_replace, when, coalesce, lit, expr

# Step 1: Trim spaces
df = df.withColumn("order_date", trim(col("order_date")))                              # Remove leading/trailing spaces from order_date

# Step 2: Normalize separators (- or . → /)
df = df.withColumn("order_date", regexp_replace(col("order_date"), "[-.]", "/"))      # Standardize date separators to "/"

# Step 3: Fill missing time only if date exists
df = df.withColumn(
    "order_date",
    when(col("order_date").isNotNull() & (~col("order_date").rlike(r"\d{1,2}/\d{1,2}/\d{4}\s\d{1,2}:\d{2}")),
         concat_ws(" ", col("order_date"), lit("00:00")))                               # Append "00:00" if time is missing
    .otherwise(col("order_date"))                                                      # Keep original value if time exists or NULL
)

# Step 4: Safely parse timestamp (invalid formats become NULL)
df = df.withColumn(
    "order_date_ts",
    expr("try_to_timestamp(order_date, 'M/d/yyyy H:mm')")                               # Convert to timestamp, invalid strings become NULL
)

# Step 5: Format timestamp as string YYYY-MM-DD HH:MM (no seconds)
df = df.withColumn(
    "order_date",
    when(col("order_date_ts").isNotNull(), F.date_format(col("order_date_ts"), "yyyy-MM-dd HH:mm"))  # Format timestamp to string without seconds
    .otherwise(None)                                                                   # Keep NULL if parsing failed
).drop("order_date_ts")                                                                 # Drop intermediate timestamp column

# updated_at

# Step 1: Trim spaces
df = df.withColumn("updated_at", trim(col("updated_at")))                              # Remove leading/trailing spaces from updated_at

# Step 2: Normalize separators (- or . → /)
df = df.withColumn("updated_at", regexp_replace(col("updated_at"), "[-.]", "/"))      # Standardize date separators to "/"

# Step 3: Fill missing time only if date exists
df = df.withColumn(
    "updated_at",
    when(col("updated_at").isNotNull() & (~col("updated_at").rlike(r"\d{1,2}/\d{1,2}/\d{4}\s\d{1,2}:\d{2}")),
         concat_ws(" ", col("updated_at"), lit("00:00")))                               # Append "00:00" if time is missing
    .otherwise(col("updated_at"))                                                      # Keep original value if time exists or NULL
)

# Step 4: Safely parse timestamp (invalid formats become NULL)
df = df.withColumn(
    "updated_at_ts",
    expr("try_to_timestamp(updated_at, 'M/d/yyyy H:mm')")                              # Convert to timestamp, invalid strings become NULL
)

# Step 5: Format timestamp as string YYYY-MM-DD HH:MM (no seconds)
df = df.withColumn(
    "updated_at",
    when(col("updated_at_ts").isNotNull(), F.date_format(col("updated_at_ts"), "yyyy-MM-dd HH:mm"))  # Format timestamp to string without seconds
    .otherwise(None)                                                                   # Keep NULL if parsing failed
).drop("updated_at_ts")                                                                 # Drop intermediate timestamp column

df.show(10, truncate=False)                                                               # Display first 10 rows without truncating columns


+---+------+-------+---------+----------------+------+----------------+
|id |name  |amount |is_active|order_date      |sex   |updated_at      |
+---+------+-------+---------+----------------+------+----------------+
|152|Ånna  |1200.0 |false    |2023-02-26 22:12|FEMALE|2023-06-01 16:02|
|415|Renée |10000.0|true     |2023-01-15 10:57|f     |2023-02-13 19:50|
|119|Judy  |1.0005 |true     |2023-06-02 17:14|M     |2023-04-06 15:32|
|365|Ivan  |-5000.0|false    |2023-02-02 21:16|f     |2023-02-15 21:07|
|30 |Bob   |NULL   |true     |NULL            |Other |2023-01-13 01:20|
|403|Frank |500.0  |true     |2023-03-31 00:58|f     |2023-04-06 05:51|
|64 |David |2000.0 |false    |2023-01-16 21:43|f     |2023-06-04 01:03|
|482|Alice |2000.5 |true     |2023-06-10 16:42|FEMALE|2023-06-10 20:17|
|442|Hannah|10000.0|false    |2023-02-08 14:43|F     |2023-02-04 07:50|
|448|Renée |2000.5 |true     |2023-05-01 17:50|f     |2023-01-22 02:21|
+---+------+-------+---------+----------------+------+----------

We remove duplicates by keeping the latest record based on the ‘updated_at’ timestamp.

In [0]:
# 7) Deduplicate

pk = ["id"]                                                                                 # Define primary key column(s) for identifying duplicates
if "updated_at" in df.columns:                                                              # Check if updated_at column exists
    w = W.partitionBy(*pk).orderBy(F.col("updated_at").desc())                               # Define window: partition by PK and order by updated_at descending
    df = (df.withColumn("_rn", F.row_number().over(w))                                        # Add row number within each PK partition
            .where(F.col("_rn") == 1)                                                       # Keep only the latest row per PK based on updated_at
            .drop("_rn"))                                                                   # Drop the temporary row number column
else:
    df = df.dropDuplicates(pk)                                                              # If updated_at doesn't exist, drop duplicate rows based on PK only


Sex values like ‘M’, ‘male’, or even ‘FEMALE’ are unified into consistent categories.

In [0]:
# 8) Normalize Categories

m = {"m":"Male","male":"Male","f":"Female","female":"Female","o":"Other","other":"Other"}  # Define mapping of various category variants to standardized values
map_expr = F.create_map([F.lit(x) for kv in m.items() for x in kv])                        # Create a Spark map literal from the dictionary for lookups
df = df.withColumn("sex", F.coalesce(F.element_at(map_expr, F.lower("sex")), F.col("sex"))) # Normalize 'sex' column using the map; fallback to original value if not in map


df.show(10,truncate=False)

+---+-----+-------+---------+----------------+------+----------------+
|id |name |amount |is_active|order_date      |sex   |updated_at      |
+---+-----+-------+---------+----------------+------+----------------+
|1  |Frank|NULL   |false    |2023-06-07 03:11|Male  |2023-06-11 02:35|
|10 |Renée|1200.0 |true     |2023-01-15 08:53|NULL  |2023-05-03 23:50|
|100|Eve  |2000.0 |false    |2023-03-15 01:46|Other |2023-03-13 01:07|
|101|Alice|5.5    |false    |2023-03-20 23:07|Male  |2023-05-31 14:31|
|103|Renée|NULL   |true     |2023-02-12 02:34|NULL  |2023-05-31 04:59|
|105|Ivan |-5000.0|false    |NULL            |NULL  |2023-04-18 22:24|
|106|Judy |NULL   |true     |2023-01-13 23:58|NULL  |2023-01-14 01:26|
|107|Alice|300.0  |false    |2023-06-07 01:30|Female|2023-04-13 04:44|
|108|Ånna |1.0005 |false    |2023-01-08 09:53|Female|2023-05-12 12:52|
|109|Bob  |2000.0 |false    |2023-05-16 11:17|NULL  |2023-05-22 07:59|
+---+-----+-------+---------+----------------+------+----------------+
only s

Extreme values are detected using the interquartile range and capped to keep them in a reasonable range.

In [0]:
# 9) Handle Outliers

q1, q3 = df.approxQuantile("amount", [0.25, 0.75], 0.01)                # Compute approximate 25th and 75th percentiles (Q1 and Q3) of 'amount$'
iqr = q3 - q1                                                              # Calculate interquartile range (IQR)
lo, hi = q1 - 1.5*iqr, q3 + 1.5*iqr                                        # Define lower and upper bounds for outliers (1.5*IQR rule)
df = df.withColumn(
    "amount_capped",
    F.when(F.col("amount") < lo, lo)                                      # Cap values below lower bound to the lower bound
     .when(F.col("amount") > hi, hi)                                      # Cap values above upper bound to the upper bound
     .otherwise(F.col("amount"))                                           # Keep values within bounds unchanged
)


df.show(10,truncate=False)

+---+-----+-------+---------+----------------+------+----------------+-------------+
|id |name |amount |is_active|order_date      |sex   |updated_at      |amount_capped|
+---+-----+-------+---------+----------------+------+----------------+-------------+
|1  |Frank|NULL   |false    |2023-06-07 03:11|Male  |2023-06-11 02:35|NULL         |
|10 |Renée|1200.0 |true     |2023-01-15 08:53|NULL  |2023-05-03 23:50|1200.0       |
|100|Eve  |2000.0 |false    |2023-03-15 01:46|Other |2023-03-13 01:07|2000.0       |
|101|Alice|5.5    |false    |2023-03-20 23:07|Male  |2023-05-31 14:31|5.5          |
|103|Renée|NULL   |true     |2023-02-12 02:34|NULL  |2023-05-31 04:59|NULL         |
|105|Ivan |-5000.0|false    |NULL            |NULL  |2023-04-18 22:24|-2987.0      |
|106|Judy |NULL   |true     |2023-01-13 23:58|NULL  |2023-01-14 01:26|NULL         |
|107|Alice|300.0  |false    |2023-06-07 01:30|Female|2023-04-13 04:44|300.0        |
|108|Ånna |1.0005 |false    |2023-01-08 09:53|Female|2023-05-12 1

We add helpful columns like the order month for easier time-based reporting

In [0]:
# 10) Add Date Ranges
df = df.withColumn("order_month", F.date_trunc("MONTH", "order_date"))

Finally, we validate that there are no missing primary keys and save the cleaned dataset as an efficient Parquet file."

In [0]:
# 11) Validate

assert df.filter(F.col("id").isNull()).count() == 0        # Ensure there are no rows with NULL in the 'id' column
assert df.count() > 0                                     # Ensure the DataFrame is not empty


And that’s it! You’ve just cleaned a complex dataset with PySpark—step by step. Don’t forget to like, share, and subscribe for more PySpark tutorials!

In [0]:
df.write.format("delta").mode("overwrite").option("mergeSchema", "true").saveAsTable("default.processed_data")

# Save the DataFrame as a Delta table named 'default.processed_data', overwriting existing data and merging the schema if needed
